In [ ]:
from typing import Dict, List, Optional, Tuple, TypedDict
from langchain_core.messages import HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import json
from datetime import datetime
from enum import Enum
import sqlite3
from datetime import datetime, timedelta
import re
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())


class ConversationStage(Enum):
    INITIAL_REQUEST = "initial_request"
    GATHERING_SERVICE_INFO = "gathering_service_info" 
    GATHERING_TIME_PREFERENCES = "gathering_time_preferences"
    GATHERING_CONTACT_INFO = "gathering_contact_info"
    CONFIRMING_DETAILS = "confirming_details"
    BOOKING_COMPLETE = "booking_complete"

class AppointmentState(TypedDict):
    # Conversation Management
    conversation_stage: ConversationStage
    messages_history: List[dict]          # Full conversation context
    missing_info: List[str]               # What we still need to collect
    
    # Information Collection (Incremental)
    seeker_request: str                   # Latest message
    service_info: Optional[dict]          # service_type, duration, special_requirements
    time_preferences: Optional[dict]      # preferred_date, preferred_time, flexibility
    seeker_contact: Optional[dict]        # name, email, phone
    location_preference: Optional[str]    # in-person vs online vs no-preference
    
    # AI Processing
    extracted_info: dict                  # Cumulative parsed information
    matched_providers: List[dict]         # Providers with required expertise
    available_slots: List[dict]           # Available time slots
    selected_slot: Optional[dict]         # Final booking choice
    
    # Final Booking
    confirmation: Optional[dict]          # Booking confirmation details
    meeting_info: Optional[dict]          # Google Meet link or location

def information_gatherer_node(state: AppointmentState) -> AppointmentState:
    """
    Main node that handles conversation flow and information collection
    """
    
    # Initialize Gemini model
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                            temperature=0,
                            max_tokens=None,
                            timeout=None,
                            max_retries=2,
                            api_key=os.environ['GEMINI_API_KEY'])
    
    # Check what information we still need
    missing_info = identify_missing_information(state)
    print("Missing Info: ", missing_info)
    if not missing_info:
        # All required info collected, move to next stage
        return {
            **state,
            "conversation_stage": ConversationStage.CONFIRMING_DETAILS,
            "missing_info": []
        }
    
    # Extract any new information from the latest message
    extracted_info = extract_information_from_message(state["seeker_request"], llm)

    print("Extracted Info: ", extracted_info)
    
    # Update state with newly extracted info
    updated_state = update_state_with_extracted_info(state, extracted_info)
    
    # Re-check missing info after extraction
    remaining_missing = identify_missing_information(updated_state)
    print("Remaining State: ", remaining_missing)
    
    if not remaining_missing:
        # All info now complete
        return {
            **updated_state,
            "conversation_stage": ConversationStage.CONFIRMING_DETAILS,
            "missing_info": []
        }
    
    # Generate appropriate follow-up question
    follow_up_message = generate_follow_up_question(updated_state, remaining_missing, llm)
    
    # Update conversation history
    new_messages = updated_state.get("messages_history", []).copy()
    new_messages.extend([
        {"role": "user", "content": state["seeker_request"]},
        {"role": "assistant", "content": follow_up_message}
    ])
    
    return {
        **updated_state,
        "messages_history": new_messages,
        "missing_info": remaining_missing,
        "conversation_stage": ConversationStage.GATHERING_SERVICE_INFO
    }

def extract_information_from_message(message: str, llm) -> Dict:
    """Extract structured information from user message using Gemini"""
    
    extraction_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert at extracting appointment booking information.
        Extract the following from the user message and return as JSON:
        
        {{
          "service_type": "haircut/doctor appointment/massage/etc or null",
          "preferred_date": "YYYY-MM-DD format or null (today is {today})",
          "preferred_time": "morning/afternoon/evening/specific time or null",
          "name": "user's name or null", 
          "contact": "phone or email or null",
          "meeting_preference": "online/in-person/no-preference or null",
          "special_requirements": "any specific requests or null"
        }}
        
        Be precise with dates. If user says "tomorrow", calculate the actual date.
        If user says "this weekend", extract as null since it's not specific.
        Return ONLY valid JSON, no additional text."""),
        ("user", "{message}")
    ])
    
    chain = extraction_prompt | llm
    
    try:
        response = chain.invoke({
            "message": message,
            "today": datetime.now().strftime("%Y-%m-%d")
        })
        
        # Clean the response and parse JSON
        content = response.content.strip()
        if content.startswith("```"):
            content = content[7:-3].strip()
        elif content.startswith("```"):
            content = content[3:-3].strip()
            
        return json.loads(content)
    except Exception as e:
        print(f"Extraction error: {e}")
        return {}

def identify_missing_information(state: AppointmentState) -> List[str]:
    """Identify what information is still missing"""
    missing = []
    
    # Check required fields
    if not state.get("service_info", {}).get("service_type"):
        missing.append("service_type")
    
    if not state.get("time_preferences", {}).get("preferred_date"):
        missing.append("preferred_date")
        
    if not state.get("seeker_contact", {}).get("name"):
        missing.append("seeker_name")
        
    if not state.get("seeker_contact", {}).get("contact"):
        missing.append("seeker_contact")
    
    return missing

def update_state_with_extracted_info(state: AppointmentState, extracted_info: Dict) -> AppointmentState:
    """Update state with newly extracted information"""
    
    updated_state = state.copy()
    
    # Update service info
    if extracted_info.get("service_type"):
        service_info = updated_state.get("service_info", {})
        service_info["service_type"] = extracted_info["service_type"]
        updated_state["service_info"] = service_info
    
    # Update time preferences
    if extracted_info.get("preferred_date") or extracted_info.get("preferred_time"):
        time_prefs = updated_state.get("time_preferences", {})
        if extracted_info.get("preferred_date"):
            time_prefs["preferred_date"] = extracted_info["preferred_date"]
        if extracted_info.get("preferred_time"):
            time_prefs["preferred_time"] = extracted_info["preferred_time"]
        updated_state["time_preferences"] = time_prefs
    
    # Update contact info
    if extracted_info.get("name") or extracted_info.get("contact"):
        contact_info = updated_state.get("seeker_contact", {})
        if extracted_info.get("name"):
            contact_info["name"] = extracted_info["name"]
        if extracted_info.get("contact"):
            contact_info["contact"] = extracted_info["contact"]
        updated_state["seeker_contact"] = contact_info
    
    return updated_state

def generate_follow_up_question(state: AppointmentState, missing_info: List[str], llm) -> str:
    """Generate contextual follow-up question based on missing information"""
    
    question_map = {
        "service_type": "What type of service are you looking for?",
        "preferred_date": "What date would work best for you?",
        "seeker_name": "Could I get your name for the appointment?",
        "seeker_contact": "How can we reach you to confirm the appointment? (email or phone)"
    }
    
    # Get the most important missing piece
    next_question = question_map.get(missing_info[0], "Could you provide more details?")
    
    # Add context based on what we already know
    context_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a friendly appointment booking assistant. 
        Generate a natural, conversational follow-up question.
        
        Current conversation context:
        - Service type: {service_type}
        - Preferred date: {preferred_date}  
        - User name: {user_name}
        
        Ask for: {missing_item}
        Base question: {base_question}
        
        Make it natural and friendly, referencing what you already know."""),
        ("user", "Generate the follow-up question")
    ])
    
    chain = context_prompt | llm
    
    try:
        response = chain.invoke({
            "service_type": state.get("service_info", {}).get("service_type", "not specified"),
            "preferred_date": state.get("time_preferences", {}).get("preferred_date", "not specified"),
            "user_name": state.get("seeker_contact", {}).get("name", "not specified"),
            "missing_item": missing_info[0],
            "base_question": next_question
        })
        return response.content
    except:
        return next_question

def route_conversation(state: AppointmentState) -> str:
    """
    Determines the next node based on current state
    """
    
    # Check if we have all required information
    missing_info = identify_missing_information(state)
    
    if missing_info:
        # Still need more information
        return "information_gatherer"
    
    # All required info collected
    if state.get("conversation_stage") == ConversationStage.CONFIRMING_DETAILS:
        return "service_matcher"
    
    # Default fallback
    return "information_gatherer"

def should_continue_gathering(state: AppointmentState) -> bool:
    """
    Helper function to check if we need more information
    """
    missing_info = identify_missing_information(state)
    return len(missing_info) > 0

# Conditional routing function for LangGraph
def conversation_router(state: AppointmentState) -> str:
    """
    Main routing function for the graph
    """
    stage = state.get("conversation_stage", ConversationStage.INITIAL_REQUEST)
    missing_info = state.get("missing_info", [])
    
    # If we still have missing required info, continue gathering
    if missing_info:
        return "gather_info"
    
    # If we have all info, move to next stage
    if stage == ConversationStage.CONFIRMING_DETAILS:
        return "match_services"
    
    # If booking is complete
    if stage == ConversationStage.BOOKING_COMPLETE:
        return "end"
    
    # Default: gather more info
    return "gather_info"

def service_matcher_node(state: AppointmentState) -> AppointmentState:
    """
    Matches seeker requirements with available providers and services
    """
    
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                            temperature=0,
                            max_tokens=None,
                            timeout=None,
                            max_retries=2,
                            api_key=os.environ['GEMINI_API_KEY'])
    
    # Extract required information from state
    service_info = state.get("service_info", {})
    time_preferences = state.get("time_preferences", {})
    location_preference = state.get("location_preference", "no-preference")
    
    # Find matching providers and services
    matched_providers = find_matching_providers(service_info, location_preference)
    
    if not matched_providers:
        return {
            **state,
            "error": "No providers found for the requested service",
            "conversation_stage": ConversationStage.BOOKING_COMPLETE
        }
    
    # Find available slots for matched providers
    available_slots = find_available_slots(
        matched_providers, 
        time_preferences,
        service_info.get("duration", 60)  # Default 60 minutes
    )
    
    if not available_slots:
        return {
            **state,
            "error": "No available slots found for your preferred time",
            "conversation_stage": ConversationStage.BOOKING_COMPLETE
        }
    
    # Use AI to select the best option
    best_match = select_best_match(matched_providers, available_slots, state, llm)
    
    return {
        **state,
        "matched_providers": matched_providers,
        "available_slots": available_slots,
        "selected_slot": best_match,
        "conversation_stage": ConversationStage.CONFIRMING_DETAILS
    }

def find_matching_providers(service_info: Dict, location_preference: str) -> List[Dict]:
    """
    Query database to find providers who offer the requested service
    """
    conn = sqlite3.connect('appointment_system.db')
    cursor = conn.cursor()
    
    service_type = service_info.get("service_type", "").lower()
    print("Service Type: ", service_type)
    
    # Query to find providers offering the service
    query = """
    SELECT DISTINCT 
        p.id, p.name, p.email, p.location, p.specialties,
        s.id as service_id, s.name as service_name, 
        s.duration_minutes, s.price
    FROM providers p
    JOIN services s ON p.id = s.provider_id
    """
        # WHERE LOWER(s.name) LIKE ? 
        # -- OR LOWER(p.specialties) LIKE ?
    
    cursor.execute(query)
    results = cursor.fetchall()
    print("Results: ")
    print(results)
    print("##"*40)
    
    providers = []
    for row in results:
        provider = {
            "provider_id": row[0],
            "provider_name": row[1],
            "provider_email": row[2],
            "location": row[3],
            "specialties": row[4],
            "service_id": row[5],
            "service_name": row[6],
            "duration_minutes": row[7],
            "price": row[8]
        }
        providers.append(provider)
    
    conn.close()
    return providers

def find_available_slots(providers: List[Dict], time_preferences: Dict, duration_minutes: int) -> List[Dict]:
    """
    Find available time slots for matched providers
    """
    conn = sqlite3.connect('appointment_system.db')
    cursor = conn.cursor()
    
    preferred_date = time_preferences.get("preferred_date")
    preferred_time = time_preferences.get("preferred_time", "any")
    
    # Calculate date range (if no specific date, check next 7 days)
    if preferred_date:
        start_date = datetime.strptime(preferred_date, "%Y-%m-%d")
        end_date = start_date + timedelta(days=1)
    else:
        start_date = datetime.now().date()
        end_date = start_date + timedelta(days=7)
    
    available_slots = []
    
    for provider in providers:
        # Query available slots for this provider
        query = """
        SELECT id, start_time, end_time, is_booked
        FROM availability_slots
        WHERE provider_id = ? 
        AND start_time >= ? 
        AND start_time <= ?
        AND is_booked = 0
        AND (end_time - start_time) >= ?
        ORDER BY start_time
        """
        
        cursor.execute(query, (
            provider["provider_id"],
            start_date.isoformat(),
            end_date.isoformat(),
            duration_minutes * 60  # Convert to seconds
        ))
        
        slots = cursor.fetchall()
        
        for slot in slots:
            slot_info = {
                "slot_id": slot[0],
                "provider_id": provider["provider_id"],
                "provider_name": provider["provider_name"],
                "service_name": provider["service_name"],
                "start_time": slot[1],
                "end_time": slot[2],
                "duration_minutes": provider["duration_minutes"],
                "price": provider["price"],
                "location": provider["location"]
            }
            
            # Filter by time preference
            if time_matches_preference(slot[1], preferred_time):
                available_slots.append(slot_info)
    
    conn.close()
    return available_slots[:10]  # Limit to top 10 options

def time_matches_preference(start_time: str, preferred_time: str) -> bool:
    """
    Check if slot time matches user preference
    """
    if preferred_time == "any":
        return True
    
    slot_hour = datetime.fromisoformat(start_time).hour
    
    if preferred_time == "morning" and 6 <= slot_hour < 12:
        return True
    elif preferred_time == "afternoon" and 12 <= slot_hour < 17:
        return True  
    elif preferred_time == "evening" and 17 <= slot_hour < 22:
        return True
    
    return False

def select_best_match(providers: List[Dict], slots: List[Dict], state: AppointmentState, llm) -> Dict:
    """
    Use AI to select the best provider/slot combination
    """
    
    selection_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an intelligent appointment scheduler. 
        Select the BEST appointment option based on these criteria:
        1. Closest to preferred time
        2. Best provider match for service type
        3. Reasonable price
        4. Location preference
        
        User preferences:
        - Service: {service_type}
        - Preferred time: {preferred_time}
        - Location preference: {location_preference}
        
        Available options:
        {options}
        
        Return the slot_id of the best option and explain why in this JSON format:
        {{
            "selected_slot_id": "slot_id_here",
            "reasoning": "Brief explanation of why this is the best choice"
        }}"""),
        ("user", "Select the best appointment option")
    ])
    
    # Format options for LLM
    options_text = ""
    for i, slot in enumerate(slots[:5], 1):  # Top 5 options
        options_text += f"""
        Option {i}:
        - Slot ID: {slot['slot_id']}
        - Provider: {slot['provider_name']}
        - Service: {slot['service_name']}
        - Time: {slot['start_time']}
        - Duration: {slot['duration_minutes']} minutes
        - Price: ${slot['price']}
        - Location: {slot['location']}
        """
    
    chain = selection_prompt | llm
    
    try:
        response = chain.invoke({
            "service_type": state.get("service_info", {}).get("service_type", ""),
            "preferred_time": state.get("time_preferences", {}).get("preferred_time", "any"),
            "location_preference": state.get("location_preference", "no-preference"),
            "options": options_text
        })
        
        # Parse response
        import json
        content = response.content.strip()
        if content.startswith("```"):
            content = content[7:-3].strip()
        
        selection = json.loads(content)
        selected_slot_id = selection["selected_slot_id"]
        
        # Find the selected slot
        for slot in slots:
            if str(slot["slot_id"]) == str(selected_slot_id):
                slot["ai_reasoning"] = selection["reasoning"]
                return slot
                
    except Exception as e:
        print(f"AI selection error: {e}")
    
    # Fallback: return first available slot
    return slots if slots else {}

def booking_node(state: AppointmentState) -> AppointmentState:
    """
    Creates the actual appointment booking
    """
    
    selected_slot = state.get("selected_slot", {})
    seeker_contact = state.get("seeker_contact", {})
    
    if not selected_slot or not seeker_contact:
        return {
            **state,
            "error": "Missing information for booking",
            "conversation_stage": ConversationStage.BOOKING_COMPLETE
        }
    
    # Create appointment in database
    appointment_id = create_appointment_record(selected_slot, seeker_contact, state)
    
    if not appointment_id:
        return {
            **state,
            "error": "Failed to create appointment",
            "conversation_stage": ConversationStage.BOOKING_COMPLETE
        }
    
    # Mark slot as booked
    update_slot_availability(selected_slot["slot_id"], True)
    
    # Generate meeting details
    meeting_info = generate_meeting_info(selected_slot, state)
    
    # Create confirmation message
    confirmation = create_confirmation_message(appointment_id, selected_slot, seeker_contact, meeting_info)
    
    return {
        **state,
        "appointment_id": appointment_id,
        "meeting_info": meeting_info,
        "confirmation": confirmation,
        "conversation_stage": ConversationStage.BOOKING_COMPLETE
    }

def create_appointment_record(slot_info: Dict, seeker_contact: Dict, state: AppointmentState) -> Optional[int]:
    """
    Create appointment record in database
    """
    conn = sqlite3.connect('appointment_system.db')
    cursor = conn.cursor()
    
    try:
        query = """
        INSERT INTO appointments (
            provider_id, seeker_name, seeker_contact, service_id, 
            slot_id, start_time, end_time, status, meeting_type, special_requirements
        ) VALUES (?, ?, ?, ?, ?, ?, ?, 'confirmed', ?, ?)
        """
        
        meeting_type = determine_meeting_type(state)
        special_requirements = state.get("service_info", {}).get("special_requirements", "")
        
        cursor.execute(query, (
            slot_info["provider_id"],
            seeker_contact["name"],
            seeker_contact["contact"],
            slot_info.get("service_id"),
            slot_info["slot_id"],
            slot_info["start_time"],
            slot_info["end_time"],
            meeting_type,
            special_requirements
        ))
        
        appointment_id = cursor.lastrowid
        conn.commit()
        return appointment_id
        
    except Exception as e:
        print(f"Database error: {e}")
        conn.rollback()
        return None
    finally:
        conn.close()

def update_slot_availability(slot_id: int, is_booked: bool) -> bool:
    """
    Update slot booking status
    """
    conn = sqlite3.connect('appointment_system.db')
    cursor = conn.cursor()
    
    try:
        cursor.execute(
            "UPDATE availability_slots SET is_booked = ? WHERE id = ?",
            (1 if is_booked else 0, slot_id)
        )
        conn.commit()
        return True
    except:
        conn.rollback()
        return False
    finally:
        conn.close()

def determine_meeting_type(state: AppointmentState) -> str:
    """
    Determine if meeting should be online, in-person, or mixed
    """
    location_pref = state.get("location_preference", "no-preference")
    service_type = state.get("service_info", {}).get("service_type", "").lower()
    
    # Some services must be in-person
    in_person_services = ["haircut", "massage", "dental", "physical therapy"]
    
    for service in in_person_services:
        if service in service_type:
            return "in-person"
    
    # Respect user preference
    if location_pref == "online":
        return "online"
    elif location_pref == "in-person":
        return "in-person"
    
    return "online"  # Default to online

def generate_meeting_info(slot_info: Dict, state: AppointmentState) -> Dict:
    """
    Generate meeting details (Google Meet link or location)
    """
    meeting_type = determine_meeting_type(state)
    
    if meeting_type == "online":
        # Generate Google Meet link (simplified)
        meet_link = f"https://meet.google.com/generated-link-{slot_info['slot_id']}"
        return {
            "type": "online",
            "meeting_link": meet_link,
            "platform": "Google Meet"
        }
    else:
        return {
            "type": "in-person", 
            "address": slot_info.get("location", "Address TBD"),
            "instructions": "Please arrive 10 minutes early"
        }

def create_confirmation_message(appointment_id: int, slot_info: Dict, seeker_contact: Dict, meeting_info: Dict) -> Dict:
    """
    Create confirmation message for the seeker
    """
    start_time = datetime.fromisoformat(slot_info["start_time"])
    
    message = f"""
    ✅ **Appointment Confirmed!**
    
    **Booking ID:** #{appointment_id}
    **Service:** {slot_info['service_name']}
    **Provider:** {slot_info['provider_name']}
    **Date & Time:** {start_time.strftime('%B %d, %Y at %I:%M %p')}
    **Duration:** {slot_info['duration_minutes']} minutes
    **Price:** ${slot_info['price']}
    
    """
    
    if meeting_info["type"] == "online":
        message += f"**Meeting Link:** {meeting_info['meeting_link']}\n"
    else:
        message += f"**Location:** {meeting_info['address']}\n"
        message += f"**Note:** {meeting_info['instructions']}\n"
    
    message += f"\nA confirmation will be sent to {seeker_contact['contact']}"
    
    return {
        "appointment_id": appointment_id,
        "message": message,
        "booking_details": slot_info,
        "contact_info": seeker_contact
    }


In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "state_db/example.db"
conn = sqlite3.connect(db_path, check_same_thread=False)

memory = SqliteSaver(conn)

def create_appointment_graph():
    """
    Creates the main appointment booking graph
    """
    
    # Create the graph
    workflow = StateGraph(AppointmentState)
    
    # Add nodes
    workflow.add_node("gather_info", information_gatherer_node)
    workflow.add_node("match_services", service_matcher_node)
    workflow.add_node("book_appointment", booking_node)
    
    # Add conditional routing
    workflow.add_conditional_edges(
        "gather_info",
        conversation_router,
        {
            "gather_info": END,
            "match_services": "match_services", # Move to service matching
            "end": END
        }
    )
    
    # Sequential flow after info gathering
    workflow.add_edge("match_services", "book_appointment")
    workflow.add_edge("book_appointment", END)
    
    # Set entry point
    workflow.set_entry_point("gather_info")
    
    return workflow.compile(checkpointer=memory)


In [ ]:
graph = create_appointment_graph()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
graph.invoke({"seeker_request": "I need a haircut"}, config={"configurable": {"thread_id": "8"}})

In [ ]:
graph.invoke({"seeker_request": "Tomorrow, 11 AM."}, config={"configurable": {"thread_id": "8"}})

In [ ]:
graph.invoke({"seeker_request": "Appointment should be under Sumit Vyas and email is smv@gmail.com"}, config={"configurable": {"thread_id": "8"}})

## New Code

In [8]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
from engine.nodes import information_gatherer_node, service_matcher_node, booking_node, conversation_router
from engine.state import AppointmentState

db_path = "state_db/example.db"
conn = sqlite3.connect(db_path, check_same_thread=False)

memory = SqliteSaver(conn)

def create_appointment_graph():
    """
    Creates the main appointment booking graph
    """
    
    # Create the graph
    workflow = StateGraph(AppointmentState)
    
    # Add nodes
    workflow.add_node("gather_info", information_gatherer_node)
    workflow.add_node("match_services", service_matcher_node)
    workflow.add_node("book_appointment", booking_node)
    
    # Add conditional routing
    workflow.add_conditional_edges(
        "gather_info",
        conversation_router,
        {
            "gather_info": END,
            "match_services": "match_services", # Move to service matching
            "end": END
        }
    )
    
    # Sequential flow after info gathering
    workflow.add_edge("match_services", "book_appointment")
    workflow.add_edge("book_appointment", END)
    
    # Set entry point
    workflow.set_entry_point("gather_info")
    
    return workflow.compile(checkpointer=memory)


In [9]:
graph = create_appointment_graph()

In [10]:
graph.invoke({"seeker_request": "I need a haircut"}, config={"configurable": {"thread_id": "9"}})

Missing Info:  ['service_type', 'preferred_date', 'seeker_name', 'seeker_contact']
Extracted Info:  {'service_type': 'haircut', 'preferred_date': None, 'preferred_time': None, 'name': None, 'contact': None, 'meeting_preference': None, 'special_requirements': None}
Remaining State:  ['preferred_date', 'seeker_name', 'seeker_contact']


{'conversation_stage': <ConversationStage.GATHERING_SERVICE_INFO: 'gathering_service_info'>,
 'messages_history': [{'role': 'user', 'content': 'I need a haircut'},
  {'role': 'assistant',
   'content': 'Okay, great! And for your haircut, what date would work best for you?'}],
 'missing_info': ['preferred_date', 'seeker_name', 'seeker_contact'],
 'seeker_request': 'I need a haircut',
 'service_info': {'service_type': 'haircut'}}